In [36]:
import os
import torch

from pyciemss.ODE.base import PetriNetODESystem
from pyciemss.ODE.events import ObservationEvent, Event
import pyciemss

In [37]:
STARTERKIT_PATH = "../test/models/starter_kit_examples/"
MIRA_PATH = "../test/models/evaluation_examples/"

filename = "CHIME-SIR/model_petri.json"
filename = os.path.join(STARTERKIT_PATH, filename)

model = PetriNetODESystem.from_mira(filename)

initial_state = torch.nn.functional.softmax(torch.randn(len(model.var_order))) * 100
initial_state = tuple(initial_state[i] for i in range(len(model.var_order)))

tspan = torch.linspace(0, 10, 100)

solution, _ = model(initial_state, tspan)

/tmp/ipykernel_24451/883036093.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  initial_state = torch.nn.functional.softmax(torch.randn(len(model.var_order))) * 100


In [38]:
observations = [ObservationEvent(torch.tensor([2.1]), {"I": torch.tensor([0.1])})]

In [33]:
pyciemss.condition(model, observations)

AttributeError: module 'pyciemss' has no attribute 'condition'

In [7]:
from mira.examples import sir